In [3]:

from gensim import corpora
from six import iteritems
import re
import codecs
import copy
from functools import reduce
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from gensim.models import LdaModel, CoherenceModel
import numpy as np
import pyLDAvis
import pyLDAvis.gensim_models
import os
import random
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

In [4]:
df_good = pd.read_csv('good_review.csv')
df_bad = pd.read_csv('bad_review.csv')
df_good = df_good[df_good['doclength']>=20]
df_bad = df_bad[df_bad['doclength']>=20]

In [6]:


#提取2018年的做为test
# df_good = df_good[df_good['year']==2018]
# df_bad = df_bad[df_bad['year']==2018]

df_good_spring = df_good[(df_good['month']==3)|(df_good['month']==4)|(df_good['month']==5)]
df_bad_spring = df_bad[(df_bad['month']==3)|(df_bad['month']==4)|(df_bad['month']==5)]

text_good_spring = df_good_spring['text']
text_bad_spring = df_bad_spring['text']



In [7]:
dictionary = corpora.Dictionary(line.lower().split() for line in text_good_spring)
four_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq <= 3]
dictionary.filter_tokens(four_ids)  # 去除只出现过一次的词
dictionary.compactify()       # 删除去除单词后的空格

text = []
for line in text_good_spring:
    text.append(line.lower().split())

class MyCorpus(object):
    def __iter__(self):
        for line in text_good_spring:
            yield dictionary.doc2bow(line.lower().split())
corpus_memory_friendly = MyCorpus()
corpus = [vector for vector in corpus_memory_friendly]  # 将读取的文档转换成语料库

LDA_models = LdaModel(corpus=corpus,id2word= dictionary, num_topics=4,
                      update_every=1,chunksize=len(corpus),passes=100, alpha='auto',random_state=42)


In [ ]:
data = pyLDAvis.gensim_models.prepare(LDA_models, corpus, dictionary)
pyLDAvis.save_html(data,'../vis_lda_good_spring_4.html')

LDA_models.save("../Lda_Model_good_spring_4")